# XGBoost Estimation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import root_mean_squared_log_error as rmsle
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

import os
import gc

## Load The Training Data


In [ ]:
# #data set from kaggle: https://www.kaggle.com/competitions/grupo-bimbo-inventory-demand/data

# load train.csv
data_path = "..\product-inventory"
filename = os.path.join(data_path, "grupo-bimbo-inventory-demand/train.csv.zip")

train = pd.read_csv(filename, 
                 usecols=['Semana', 'Producto_ID', 'Cliente_ID', 'Demanda_uni_equil'])

# rename columns
train = train.rename(columns={  'Semana': 'Week_num',
                                'Cliente_ID': 'Client_ID',
                                'Demanda_uni_equil': 'adjusted_demand',
                                'Producto_ID': 'Product_ID'})

# # define client-product ID
# train['ID'] = train.groupby(['Client_ID', 'Product_ID']).ngroup()
# unique_ids = train['ID'].unique()

# # Define the fraction of IDs to sample
# fraction = 0.1  # sample 10% of the IDs

# # Calculate the number of IDs to sample
# sample_size = int(len(unique_ids) * fraction)

# rng = np.random.default_rng(4325252122)

# # Choose a random sample of IDs
# sampled_ids = np.random.choice(unique_ids, size=sample_size, replace=False)

# # Filter the DataFrame to keep all rows with the sampled IDs
# train = train[train['ID'].isin(sampled_ids)]
# train = train.drop(columns='ID')
# print(len(train))

In [ ]:
# duplicates client-product-week observation -> take the average as adjusted demand
train = train.groupby(['Client_ID', 'Product_ID', 'Week_num'], as_index=False).agg({'adjusted_demand': 'sum'})

# create the target dataset and training dataset
# identify the last observation of each client-product
last_week = train.groupby(['Client_ID', 'Product_ID'], as_index=False).agg({'Week_num':'max'})
train  = train.merge(right = last_week,
                    how='left',
                    on=['Client_ID', 'Product_ID'],
                    suffixes=['','_max'])

del last_week

# target dataset
val = train.loc[train['Week_num'] == train['Week_num_max']]
val = val.drop(columns=['Week_num_max'])

# train dataset
train = train.loc[train['Week_num'] != train['Week_num_max']]
train = train.drop(columns=['Week_num_max'])


In [ ]:
print(len(train), len(val))
print('\n')
print(train.columns)
print(val.columns)

In [ ]:
#create a dataframe of aggregate statistics for each client
client_stats = train.groupby(['Client_ID'], as_index=False).agg({'Product_ID':'nunique', 'adjusted_demand':['mean', 'median']})
client_stats.columns = ['Client_ID', 'Products', 'adj_dem_mean', 'adj_dem_median']
client_stats['adj_dem_mean'] = client_stats['adj_dem_mean'].round(2)
client_stats['adj_dem_median'] = client_stats['adj_dem_median'].astype(int)

#create a dataframe of aggregate statistics for each product
product_stats = train.groupby(['Product_ID'], as_index=False).agg({'Client_ID':'nunique', 'adjusted_demand':['mean', 'median']})
product_stats.columns = ['Product_ID', 'Clients', 'adj_dem_mean', 'adj_dem_median']
product_stats['adj_dem_mean'] = product_stats['adj_dem_mean'].round(2)
product_stats['adj_dem_median'] = product_stats['adj_dem_median'].astype(int)
product_stats['median_pct'] = product_stats['adj_dem_median'].rank(pct=True, method='average')

#create a dataframe of aggregate client-product information i.e. removing the time dimension
train = train.groupby(['Client_ID', 'Product_ID'], as_index=False).agg({'Week_num':'nunique', 'adjusted_demand':['mean', 'median', 'min', 'max']})
train.columns = ['Client_ID', 'Product_ID', 'num_weeks', 'adj_dem_mean', 'adj_dem_median', 'adj_dem_min', 'adj_dem_max']

In [ ]:
train = train.merge(right=client_stats,
                    how='left',
                    on='Client_ID',
                    suffixes=('', '_c'))
del client_stats

train = train.merge(right=product_stats,
                    how='left',
                    on='Product_ID',
                    suffixes=('','_p'))
del product_stats

train = train.merge(right=val[['Client_ID', 'Product_ID', 'adjusted_demand']],
                how='left',
                on=['Product_ID', 'Client_ID'])
train.columns

In [ ]:
features = ['num_weeks','adj_dem_median', 'adj_dem_min', 'adj_dem_max',
            'Products','adj_dem_mean_c', 'adj_dem_median_c',
            'Clients', 'adj_dem_mean_p', 'adj_dem_median_p', 'median_pct']
target = ['adjusted_demand']

## Grid Search

In [ ]:
param_grid = {"max_depth":    [3, 4, 5],
              "n_estimators": np.arange(100,800,100),
              "learning_rate": [0.01, 0.1, 1]}

In [ ]:
xgb_reg = xgb.XGBRegressor()

In [ ]:
# Note:  this took about 3 minutes to run on my 2023 MacBook Pro.
search = GridSearchCV(xgb_reg, param_grid, cv=5, scoring = 'neg_root_mean_squared_log_error').fit(train[features], train[target])

print("The best hyperparameters are ",search.best_params_)

In [ ]:
# dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=False)
bst = xgb_reg.fit(X_train, y_train)

In [ ]:
# predict the demand
train['pred'] = xgb_reg.predict(X_train)
train['pred'] = np.maximum(0, train['pred']).round(2)
train[target+['pred', 'Client_ID', 'Product_ID']+features]

## Load The Testing Data

In [ ]:
# load test.csv
data_path = "..\product-inventory"
filename = os.path.join(data_path, "grupo-bimbo-inventory-demand/test.csv.zip")

test = pd.read_csv(filename, 
                 usecols=['id', 'Producto_ID', 'Cliente_ID'])
# 
# rename columns
test = test.rename(columns={'Cliente_ID': 'Client_ID',
                            'Producto_ID': 'Product_ID'})

In [ ]:
test = test[['id','Client_ID', 'Product_ID']].merge(right=train[['Client_ID', 'Product_ID', 'pred']], 
                                                    how='left', 
                                                    on=['Client_ID', 'Product_ID'])
test = test.sort_values(by=['Client_ID', 'Product_ID']).reset_index(drop=True)
test

In [ ]:
test['pred'].isna().mean()

## Out-of-Sample Prediction
There are a few cases where the client-product combo is not present in the training data. I fill in these observations' missing values. 
### 1. Existing Products
This case consists of two possible scanarios:
* New Client
* Existing Client but a new combo

In both scenarios, the in-sample estimation can not predict the demand. We use the average product demand in weeks 3-9 as our prediction.
In the 2nd scenario, our prediction model has some shortcomings. For example, our measure does not take into account that a client might have a low demand for a new product. On the other hand, if we use the client's average demand as our prediction, it does not take into account the variation in products' demand. As a first pass, we use 'average prodct demand' as the prediction


In [ ]:
# use product's average demand in week 3-9 as a prediction for the new client.
# adj_dem_mean_p contains existing product with new client, and a prediction for the client's demand.

#create a dataframe of aggregate statistics for each product
testagg = train.groupby(['Product_ID'], as_index=False).agg({'adj_dem_mean_p':'mean'})

test = test.merge(right=testagg, 
                  how='left', 
                  on='Product_ID')

del testagg
test['pred'] = test['pred'].fillna(test['adj_dem_mean_p'])
print('Share of Missing Preiction:', test['pred'].isna().mean())

test = test.drop(columns='adj_dem_mean_p')

### 2. Existing Clients

In [ ]:
# use client's average demand in week 3-9 as a prediction for new product.
#pred_1 contains existing clients with new products, and a prediction for the product.

#create a dataframe of aggregate statistics for each product
testagg = train.groupby(['Client_ID'], as_index=False).agg({'adj_dem_mean_c':'mean'})
test = test.merge(right=testagg, 
                  how='left', 
                  on='Client_ID')

test['pred'] = test['pred'].fillna(test['adj_dem_mean_c'])
print('Share of Missing Preiction:', test['pred'].isna().mean())

del testagg
test = test.drop(columns='adj_dem_mean_c')
# WATCH OUT: This replaces missing values for existing clients and existing products with the client's average demand. 
# THEY SHOULD BE REPLACED WITH THE ACTUAL PREDICTION

In [ ]:
train.columns

### 3. New Clients, New Products
The intersection of new clients and new product in the test data. Here the first guess is the average demand for all product across all weeks.


In [ ]:
test['pred'] = test['pred'].fillna(train['adj_dem_mean'].mean())
print('Share of Missing Preiction:', test['pred'].isna().mean())

In [ ]:
test

In [ ]:
output = test[['id', 'pred']]
output = output.rename(columns={'pred': 'Demanda_uni_equil'})

data_path = "..\product-inventory"
filename = os.path.join(data_path, "prediction_3.csv")
output.to_csv(filename, index=False)